In [2]:
#------------ extract relevant conference in a file [paperid, conferenceid] -------------#
# !cat ../data/paperconferenceid.tsv | grep '1163450153\|1195049314\|1166315290\|2754292954\|1171345118' > ../data/HCI_paperids.tsv

cat: ../data/paperconferenceid.tsv: No such file or directory


In [34]:
#------------ paper year -------------# data source:/data/pcsdata/selected_MAG_metadata/paperyear.tsv
from itertools import chain
import pandas as pd
# data_HCI = list(chain(*pd.read_csv('../data/HCI_paperids.tsv',sep=',').values.tolist()))
data_HCI = pd.read_csv('../data/HCI_paperids.tsv',sep='\t')
data_HCI.head(10)

,paperid,confid
0,2243662716,1163450153
1,2244265604,1163450153
2,2244428935,1163450153
3,2244483578,1163450153
4,2244508988,1163450153
5,2336371700,1163450153
6,2244624262,1163450153
7,2244786198,1163450153
8,2321239069,1163450153
9,2245049727,1163450153


In [ ]:
results = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
ori_file = open('../data/paperyear.tsv')
conf_name_map = {1163450153:'CHI', 1195049314:'CSCW', 1171345118:'UBI', 1166315290:'UIST'}
# conf_name_map could also load by category
# data_category = pd.read_csv('../data/category.tsv',sep=',')
# data_category = data_category.loc[data_category['category']=="AI"]
# for row in data_category.iterrows():
#     conf_name_map[row[0]] = row[1]

iter_f = iter(ori_file)
first_line = True
for row in iter_f:
    if first_line:
        first_line = False
        continue
    paperid_str = str(row).split()[0] # paperid
    paperrow_str = str(row).split() # paper row
    paperid = int(paperid_str)
    if paperid in data_HCI['paperid'].values:
        results['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))
for name, result in results.items():
    result_pd = pd.DataFrame(data=result, columns=['paperid', 'year'])
    result_pd.to_csv('paperyear_' + str(name) + '.tsv')
print(">>> done")

In [ ]:
#------------ paper titles -------------# data source: /data4/pcs/papertitle.tsv
import os
from itertools import chain
import pandas as pd

results_title = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
ori_pd = pd.read_csv('../data/papertitle.tsv', skiprows=1)
pd.set_option("display.max_colwidth", 100000)

for row in ori_pd.iterrows():
    paperid_str = str(row[1]).split('\\t')[1].split()[1] # paperid
    paper_title = str(row[1]).split('\\t')[2].split('\n')[0]
    paperrow_str = [paperid_str, paper_title]
    paperid = int(paperid_str)
    if paperid in data_HCI['paperid'].values:
        results['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))
        
for name, result in results_title.items():
    result_pd = pd.DataFrame(data=result, columns=['paperid', 'title'])
    result_pd.to_csv('papertitle_' + str(name) + '.tsv')
print(">>> done")

In [ ]:
#------------ paper author order -------------#
from itertools import chain
import pandas as pd

results_author = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
ori_paperauthor = pd.read_csv('../data/paperauthororder.tsv',skiprows=1)
for row in ori_paperauthor.iterrows():
    paperid_str = str(row[1]).split('    ')[1].split('\\t')[0] # paperid
    paperrow_str = str(row[1]).split('    ')[1].split('\n')[0].split('\\t') # paper row
    paperid = int(paperid_str)

    if paperid in data_HCI['paperid'].values:
        results_author['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))

for name, result in results_author.items():
    result_pd = pd.DataFrame(data=result, columns=['paperid', 'authorid', 'authororder'])
    result_pd.to_csv('paperauthororder_' + str(name) + '.tsv')
print(">>> done")

In [ ]:
#------------ paper affiliations -------------#
import os
from itertools import chain
import pandas as pd

results_affiliation = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
ori_pd = pd.read_csv('../data/paperauthoridaffiliationname.tsv', sep='\t', skiprows=1)
pd.set_option("display.max_colwidth", 100000)

for row in ori_pd.iterrows():
    paperid_str = str(row[1]).split('\n')[0].split()[1]
    authorid = str(row[1]).split('\n')[1].split()[1]
    affiliationname = str(row[1]).split('\n')[2].split()[1]
    paperrow_str = [paperid_str, authorid, affiliationname]
    paperid = int(paperid_str)

    if paperid in data_HCI['paperid'].values:
        results_aff['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))

for name, result in results_affiliation.items():
    result_pd = pd.DataFrame(data=result, columns=['paperid', 'authorid', 'affiliationname'])
    result_pd.to_csv('authoraffname_' + str(name) + '.tsv')
print(">>> done")

In [ ]:
#------------ paper citations -------------# data source: /data4/pcs/papercitations
import os
from itertools import chain
import pandas as pd

results_citing = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
results_cited = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}

# all citation raw file are listed in papercitations directory
files = os.listdir("../data/papercitations")
for file in files:
    firstline = True
    paper_citation_file = open(os.path.join("../data/papercitations", file))
    iter_f = iter(paper_citation_file)
    for row in iter_f:
        if firstline:
            firstline = False
            continue
        if len(str(row).split()) < 2:
            continue
        citing_paperid_str = str(row).split()[0] # citingpaperid
        cited_paperid_str = str(row).split()[1]
        paperrow_str = [citing_paperid_str, cited_paperid_str]
        paperid = int(citing_paperid_str)
        paperid2 = int(cited_paperid_str)

        if paperid in data_HCI['paperid'].values:
            results_citing['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))

        if paperid2 in data_HCI['paperid'].values:
            results_cited['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid2]['confid'].values)])].append(list(paperrow_str))

for name, result in results_citing.items():
    result_pd = pd.DataFrame(data=result, columns=['citingpaperid', 'citedpaperid'])
    result_pd.to_csv('paperciting_' + str(name) + '.tsv')

for name, result in results_cited.items():
    result_pd = pd.DataFrame(data=result, columns=['citingpaperid', 'citedpaperid'])
    result_pd.to_csv('papercited_' + str(name) + '.tsv')

print(">>> done")

In [ ]:
#------------ paper citations science -------------# data source: /data/pcsdata/pcs.tsv
from itertools import chain
import pandas as pd

results_pcs = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
ori_file = open('../data/pcs.tsv')
iter_f = iter(ori_file)
first_line = True
for row in iter_f:
    if first_line:
        first_line = False
        continue
    paperrow_str = str(row).split() # paper row
    paperid_str = paperrow_str[2]
    paperid = int(paperid_str)
    if paperid in data_HCI['paperid'].values:
        results_pcs['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))

for name, result in results_pcs.items():
    result_pd = pd.DataFrame(data=result, columns=['reftype', 'confscore', 'paperid', 'patent'])
    result_pd.to_csv('papercitationscience_' + str(name) + '.tsv')

In [40]:
#------------ patent inventors -------------# data source:pcs/patent_dblpdata/patent_inventor.tsv
from itertools import chain
import pandas as pd
patent_data_HCI = pd.read_csv('../data/papercitationscience_result.tsv',sep='\t')
patent_data_HCI = patent_data_HCI.merge(data_HCI, left_index=True, right_index=True)
patent_data_HCI.head(5)

,",reftype,confscore,paperid,patent",paperid,confid
0,"0,app,9,2134161165,10001804",2243662716,1163450153
1,"1,app,9,1967451823,10001885",2244265604,1163450153
2,"2,app,9,1967451823,10001888",2244428935,1163450153
3,"3,app,10,2005198142,10001888",2244483578,1163450153
4,"4,app,10,2152754623,10001897",2244508988,1163450153


In [ ]:
results_inventors = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}

ori_patentinventors = pd.read_csv('../data/patent_inventor.tsv',skiprows=1)
for row in ori_patentinventors.iterrows():
    patent_id = str(row[1]).split('    ')[1].split('\\t')[0]
    patentinventors_row = str(row[1]).replace("\"","").split('    ')[1].split('\n')[0].split('\\t')

    if paperid in data_HCI['paperid'].values:
        results_inventors['result_{}'.format(conf_name_map[int(patent_data_HCI.loc[patent_data_HCI['paperid']==paperid]['confid'].values)])].append(list(patentinventors_row))
        
for name, result in results_inventors.items():
    result_pd = pd.DataFrame(data=result, columns=['patent_id', 'inventor_id', 'location_id'])
    result_pd.to_csv('patentinventors_' + str(name) + '.tsv')
print(">>> done")

In [ ]:
#------------ patent abstract -------------# data source: /data4/pcs/abstract/brf_sum_text.tsv
from itertools import chain
import pandas as pd
results_abstract = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}

ori_patent_abstract = pd.read_csv('../data/brf_sum_text.tsv', sep='\t', skiprows=1)
pd.set_option("display.max_colwidth", 100000)
for row in ori_patent_abstract.iterrows():
    uuid = str(row[1]).split('\n')[0].split()[1].strip("\n")
    patent_id = str(row[1]).split('\n')[1].split()[1].strip("\n")
    abstract = str(row[1]).split('\n')[2].split('         ')[1]
    patent_abstract_row = [uuid, patent_id, abstract]
    
    if paperid in data_HCI['paperid'].values:
        results_abstract['result_{}'.format(conf_name_map[int(patent_data_HCI.loc[patent_data_HCI['paperid']==paperid]['confid'].values)])].append(list(patent_abstract_row))

for name, result in results_inventors.items():
    result_pd = pd.DataFrame(data=result, columns=['uuid', 'patent_id', 'abstract'])
    result_pd.to_csv('patent_abstract_' + str(name) + '.tsv')
print(">>> done")

In [48]:
#------------ dblp title match -------------#
# -*- coding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import pandas as pd
import json

results_dblp_title_match = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
conf_name_map = {1163450153:'CHI', 1195049314:'CSCW', 1171345118:'UBI', 1166315290:'UIST'}
paper = []
for line in open('dblp_papers_with_s2_abstracts.jsonlist', 'r'):
    paper.append(json.loads(line))
df = pd.DataFrame(paper)

data_papers_HCI = {'data_CHI':[], 'data_UBI', 'data_CSCW':[], 'data_UIST':[]}
for conf_id, conf_name in conf_name_map.items():
    data_papers_HCI['data_{}'.format(conf_name)] = df[(df['dblp_venue'] == conf_name)][['dblp_title', 'dblp_venue', 's2_abstract']]
    data_papers_HCI['data_{}'.format(conf_name)]['papertitle'] = [title.encode('utf-8').strip().lower() for title in data_papers_HCI['data_{}'.format(conf_name)]['dblp_title']]
    data_papers_HCI['data_{}'.format(conf_name)]['papertitle'].to_csv('dblp_{}_title.tsv'.format(conf_name))
    data_papers_HCI['data_{}'.format(conf_name)]['s2_abstract'].to_csv('dblp_{}_abstract.tsv'.format(conf_name))

1163450153
1195049314
1171345118
1166315290


In [ ]:
# -*- coding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import pandas as pd

for conf_name in ['CHI', 'CSCW', 'UBI', 'UIST']:
    hci_tilte = pd.read_csv("../data/papertitle_{}.tsv".format(conf_name), sep = ",")

    hci_tilte['papertitle'] = [title.encode('utf-8').strip().lower() for title in hci_tilte['title']]

    from fuzzywuzzy import process

    # matched = 'Technology heirlooms?: considerations fo passing down and inheriting digital materials.'
    # print(papers['papertitle'].iloc[0])
    # highest = process.extractOne(str(papers['papertitle'].iloc[0]), hci_title['papertitle'])
    # print(highest)

    result = []
    with open('dblp_{}_title.tsv'.format(conf_name), "r") as title_f:
        for line in title_f:
            paper_idx = line[0:line.index(',')]
            paper_title = line[line.index(',') + 1:]
            highest = process.extractOne(paper_title, hci_tilte['papertitle'])
            print(highest)
            try:
                output = [highest[0], highest[1], highest[2], paper_idx]
                result.append(list(output))
            except:
                print(">>> error")
        result_pd = pd.DataFrame(data=result, columns=['papertitle', 'score', 'paperidx', 'dblppaperidx'])
        result_pd.to_csv('matched_{}.tsv'.format(conf_name))
